In [66]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm


import warnings

# Suppress the warning
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv()